# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [96]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [97]:
path = 'output_data/cities.csv'
map_df = pd.read_csv(path)
map_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Shahr-e Bābak,30.1165,55.1186,74.71,19,15,5.26,IR,2021-09-19 19:56:38
1,1,Comodoro Rivadavia,-45.8667,-67.5000,52.86,55,7,4.27,AR,2021-09-19 20:00:01
2,2,Punta Arenas,-53.1500,-70.9167,50.11,57,75,31.07,CL,2021-09-19 19:51:22
3,3,Kaitangata,-46.2817,169.8464,41.32,92,46,5.84,NZ,2021-09-19 19:54:29
4,4,Vaini,-21.2000,-175.2000,71.76,88,40,9.22,TO,2021-09-19 19:51:49
...,...,...,...,...,...,...,...,...,...,...
550,550,Porto-Novo,6.4965,2.6036,77.47,85,60,6.46,BJ,2021-09-19 20:06:25
551,551,Stromness,58.9650,-3.2960,55.65,90,10,6.15,GB,2021-09-19 20:06:25
552,552,Fuyang,32.9000,115.8167,62.24,95,100,9.91,CN,2021-09-19 20:06:26
553,553,Lompoc,34.6391,-120.4579,88.11,63,1,19.57,US,2021-09-19 20:02:39


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [98]:
# Gmaps configuration 
gmaps.configure(api_key=g_key)
# Lat and Lng stored as locations
locations = map_df[["Latitude", "Longitude"]].astype(float)
# Humidity stored as weight
weight = map_df["Humidity"].astype(float)
# Map Style
with open('resources/map.json') as map:
    geometry = json.load(map)
# Create google map figure
fig = gmaps.figure(zoom_level=2, center=(40,-100), layout={'width': '1050px', 'height': '700px', 'margin': '0 auto 0 auto'})
# # GeoJSON
# geojson_layer = gmaps.geojson_layer(geometry)
# Add heat map layer
heat_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, max_intensity=100, point_radius=2)
# Initialize Map
fig.add_layer(heat_layer)
# Show
fig

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', width='1050px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [99]:
# Temp 60-80 F
# Cloudiness > 30%
# Wind Speed < 10mph
# Humidity < 20%
map_df = hotel_df = map_df[(map_df['Max Temperature'] > 60) & (map_df['Max Temperature'] < 80) & (map_df['Wind Speed'] <= 10) & (map_df['Cloudiness'] >= 30) & (map_df['Humidity'] < 60)]
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
24,24,Faya,18.3851,42.4509,70.54,21,95,8.77,SA,2021-09-19 20:00:25
67,67,Chapais,49.7834,-74.8492,62.28,41,72,5.01,CA,2021-09-19 20:01:00
131,131,São Roque,-23.5292,-47.1353,71.37,54,38,9.17,BR,2021-09-19 20:01:36
240,240,Sayyan,15.1718,44.3244,66.56,34,55,4.83,YE,2021-09-19 20:03:08
286,286,Menongue,-14.6585,17.6910,68.29,54,33,5.35,AO,2021-09-19 20:03:34
291,291,Kumul,42.8000,93.4500,72.45,20,34,1.81,CN,2021-09-19 20:03:36
453,453,Bačko Petrovo Selo,45.7081,20.0753,65.12,50,58,7.27,RS,2021-09-19 20:05:15
462,462,Buin,-33.7333,-70.7500,62.40,55,59,4.85,CL,2021-09-19 20:01:42
483,483,General Roca,-39.0333,-67.5833,68.40,29,59,8.61,AR,2021-09-19 20:05:32


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [100]:
# Add url for Google Places API
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Set params
params = {
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}
# Loop through 'hotel_df'
for index, row in hotel_df.iterrows():
    # Location params
    lat = row['Latitude']
    lon = row['Longitude']
    # Updates location params whilst looping
    params['location'] = f'{lat},{lon}'
    # Make api call
    response = requests.get(url, params=params).json()
    # Try to append the hotels requested to appropriate columns
    try:
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except(KeyError, IndexError):
        print('Missing Hotel Info..Skipping...')
hotel_df    

Missing Hotel Info..Skipping...


C:\Users\cheye\anaconda3\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\cheye\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Missing Hotel Info..Skipping...
Missing Hotel Info..Skipping...


,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
24,24,Faya,18.3851,42.4509,70.54,21,95,8.77,SA,2021-09-19 20:00:25,NaN
67,67,Chapais,49.7834,-74.8492,62.28,41,72,5.01,CA,2021-09-19 20:01:00,Hôtel Opémiska
131,131,São Roque,-23.5292,-47.1353,71.37,54,38,9.17,BR,2021-09-19 20:01:36,Villa Rossa Hotel
240,240,Sayyan,15.1718,44.3244,66.56,34,55,4.83,YE,2021-09-19 20:03:08,NaN
286,286,Menongue,-14.6585,17.6910,68.29,54,33,5.35,AO,2021-09-19 20:03:34,Ritz Lauca
291,291,Kumul,42.8000,93.4500,72.45,20,34,1.81,CN,2021-09-19 20:03:36,Yushang Hotel
453,453,Bačko Petrovo Selo,45.7081,20.0753,65.12,50,58,7.27,RS,2021-09-19 20:05:15,NaN
462,462,Buin,-33.7333,-70.7500,62.40,55,59,4.85,CL,2021-09-19 20:01:42,Motel INSIDE
483,483,General Roca,-39.0333,-67.5833,68.40,29,59,8.61,AR,2021-09-19 20:05:32,Estepa Hotel


In [101]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [102]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations, info_box_content=hotel_info)
# Add markers
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='700px', margin='0 auto 0 auto', width='1050px'))